<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_192WinLen_LossOptimised_With_Volume_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1784, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1784 (delta 0), reused 0 (delta 0), pack-reused 1781
Receiving objects: 100% (1784/1784), 81.45 MiB | 21.48 MiB/s, done.
Resolving deltas: 100% (851/851), done.
Checking out files: 100% (1940/1940), done.


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_data.py

100% 5/5 [00:01<00:00,  3.08it/s]
{0: 'MCD', 1: 'BEEM', 2: 'XXII', 3: 'RWB', 4: 'PSTV'}
{'MCD': 0, 'BEEM': 1, 'XXII': 2, 'RWB': 3, 'PSTV': 4}
train labels:  (10908, 192)
train inputs:  (10908, 192, 5)
  0% 0/1151 [00:00<?, ?it/s]Company Name MTVR not in Train Data. Hence skipping
Company Name BILL not in Train Data. Hence skipping
Company Name ULTA not in Train Data. Hence skipping
Company Name PHUN not in Train Data. Hence skipping
Company Name COPX not in Train Data. Hence skipping
Company Name W not in Train Data. Hence skipping
Company Name BIIB not in Train Data. Hence skipping
Company Name BTU not in Train Data. Hence skipping
Company Name KO not in Train Data. Hence skipping
Company Name ALXN not in Train Data. Hence skipping
Company Name AVGO not in Train Data. Hence skipping
Company Name LK not in Train Data. Hence skipping
Company Name KSM not in Train Data. Hence skipping
Company Name PLD not in Train Data. Hence skipping
Company Name BKNG not in Train Data. Hence skipping
C

In [ ]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [ ]:
print(index_to_company)
print(company_to_index)

{0: 'MCD', 1: 'BEEM', 2: 'XXII', 3: 'RWB', 4: 'PSTV'}
{'MCD': 0, 'BEEM': 1, 'XXII': 2, 'RWB': 3, 'PSTV': 4}


In [ ]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [ ]:
NUM_TRAIN_SAMPLES = 10000
NUM_TEST_SAMPLES  = 3800

# stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
# stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

# stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
# stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

stock_inputs_trimmed = stock_inputs[:,:,:]
stock_labels_trimmed = stock_labels[:,:]

stock_test_inputs_trimmed = stock_test_inputs[:,:,:]
stock_test_labels_trimmed = stock_test_labels[:,:]

In [ ]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (10908, 192, 5)
stock_labels_trimmed shape (10908, 192)
stock_test_inputs_trimmed shape (4236, 192, 5)
stock_test_labels_trimmed shape (4236, 192)


In [ ]:
import pandas as pd

pd.DataFrame(stock_inputs_trimmed[:, 1, :]).to_csv("stock_inputs_trimmed.csv")

In [ ]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, len(stock_inputs_trimmed)):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, len(stock_test_inputs_trimmed)):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])




In [ ]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))


[0.0, 1.0, 2.0, 3.0, 4.0]
5
[0.0, 1.0, 2.0, 4.0]
4


In [ ]:

params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':256,
    'cov_dim': 4,
    'lstm_hidden_dim': 64,
    'lstm_layers':4 ,
    'window_size':192,
    'batch_size': 96,
    'learning_rate': 1e-3,
    'epochs':20,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'num_gmms': 1,
    'mu_sigma_dim': 1  # each datapoint is one-d 
}

print(params)

{'num_classes': 5, 'embedding_dim': 256, 'cov_dim': 4, 'lstm_hidden_dim': 64, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 96, 'learning_rate': 0.001, 'epochs': 20, 'num_test_samples': 3800, 'num_train_samples': 10000, 'conditioning_period': 168, 'prediction_period': 24, 'num_gmms': 1, 'mu_sigma_dim': 1}


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        #Deleting Volume as covariate
        # x = np.delete(self.data[index], 1, axis=1)
        x = self.data[index]
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2
        
class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
        # x = np.delete(self.data[index], 1, axis=1)
        x = self.data[index]
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2
        

In [ ]:
def unwindow_sequence(dataset):
  company_data = {}

  seq_id = 0
  company_index_cov_id = -1

  for cov_ip, cov_op in dataset:

    comp_index = cov_ip[seq_id][company_index_cov_id]
    company_name = index_to_company[comp_index.item()]

    # IP Shape (Time, Dim)
    # OP Shape (Time, 1) # One covariate (open price) per time instant
    if company_data.get(company_name, None) is None:
      company_data[company_name] = [cov_ip, cov_op]
      
    else:
      # windows are created with stride 1
      # so we should be appending only the last element in the sequence
      cov_ip_last = cov_ip[-1, :].unsqueeze(0)
      cov_op_last = cov_op[-1].unsqueeze(0)

      # 0 --> Covariates
      # 1 --> Outputs, aka stock price
      company_data[company_name][0] = torch.cat((company_data[company_name][0], cov_ip_last), dim=0)
      company_data[company_name][1] = torch.cat((company_data[company_name][1], cov_op_last), dim=0)

  return company_data

In [ ]:
train_data = TrainDataset()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)

train_data_unwindowed = unwindow_sequence(train_data)
test_data_unwindowed = unwindow_sequence(test_data)

In [ ]:
print(train_data_unwindowed.keys())
print(len(train_data_unwindowed))

dict_keys(['MCD', 'BEEM', 'XXII', 'RWB', 'PSTV'])
5


In [ ]:
print("train_data_unwindowed", train_data_unwindowed['MCD'][0].shape) 

train_data_unwindowed torch.Size([3920, 5])


In [ ]:
ip_data = train_data_unwindowed['MCD'][0]
ip_data_price = ip_data[:, 0]
print("---ip data price ", ip_data_price)


---ip data price  tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4525,  0.4282,  0.4346])


In [ ]:
import plotly.express as px

df = pd.DataFrame(data=ip_data_price.detach().cpu())
fig = px.histogram(x=df[0],range_x=[df.min(),df.max()])
fig.show()
# plt.hist(ip_data_price, bins='auto')  # arguments are passed to np.histogram
# plt.title("Histogram with 'auto' bins")

In [ ]:
print(test_data_unwindowed.keys())
print(len(test_data_unwindowed))
# Some Companies are there in the Train Set, But not in the Test Set. 
# However, all companies in the Test Set is there in the Train Set

dict_keys(['XXII', 'BEEM', 'MCD', 'PSTV'])
4


In [ ]:
import plotly.express as px

def plot_output(company_data, company_name):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[company_name][1]), num=len(company_data[company_name][1]))
  fig = px.line(x=x, y=company_data[company_name][1])
  fig.show()
  print("op", company_data[company_name][1])

def plot_input_covariate(company_data, company_name, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[company_name][0].shape[0], num=len(company_data[company_name][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[company_name][0][:,covariate_index])
  fig.show()
  print("ip", company_data[company_name][0][:,covariate_index])

In [ ]:
# SANITY Check One: Output Label (open price) is shifted by one element of input (the ip opening price)
plot_output(train_data_unwindowed, 'MCD')
plot_input_covariate(train_data_unwindowed, 'MCD', 0)

op tensor([-0.6790, -0.7051, -0.6279,  ...,  0.4282,  0.4346,  0.4358])


ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4525,  0.4282,  0.4346])


In [ ]:

# SANITY Check TWO: Train and Test data are contiguous
plot_input_covariate(train_data_unwindowed, 'MCD', 0)
plot_input_covariate(test_data_unwindowed, 'MCD', 0)


ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4525,  0.4282,  0.4346])


ip tensor([0.4295, 0.4304, 0.4237,  ..., 1.7686, 1.7638, 1.7802])


In [ ]:
# SANITY Check TWO for Output Labels: Train and Test data are contiguous

plot_output(train_data_unwindowed, 'MCD')
plot_output(test_data_unwindowed, 'MCD')

op tensor([-0.6790, -0.7051, -0.6279,  ...,  0.4282,  0.4346,  0.4358])


op tensor([0.4304, 0.4237, 0.4167,  ..., 1.7638, 1.7802, 1.7692])


In [ ]:
plot_input_covariate(train_data_unwindowed, 'MCD', 0)  # Opening Price
plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Volume
plot_input_covariate(train_data_unwindowed, 'MCD', 2)  # Day
plot_input_covariate(train_data_unwindowed, 'MCD', 3)  # Month
plot_input_covariate(train_data_unwindowed, 'MCD', 4)  # Company ID


ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4525,  0.4282,  0.4346])


ip tensor([-0.4795, -1.5838,  0.8233,  ...,  0.4954, -0.0370, -0.2441])


ip tensor([-1.6811, -1.5670, -1.4528,  ...,  1.5150, -1.6811, -1.5670])


ip tensor([ 1.2869,  1.2869,  1.2869,  ..., -0.4512, -0.1615, -0.1615])


ip tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [ ]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[window_id][1]), num=len(company_data[window_id][1]))
  fig = px.line(x=x, y=company_data[window_id][1])
  fig.show()
  print("op", company_data[window_id][1])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[window_id][0].shape[0], num=len(company_data[window_id][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][0][:,covariate_index])
  fig.show()
  print("ip", company_data[window_id][0][:,covariate_index])


In [ ]:
plot_output_windowed(test_data, 0)
plot_input_covariate_windowed(test_data, 0, 0)

op tensor([1.2605, 1.2298, 1.1502, 1.1718, 1.0373, 1.0834, 0.9776, 0.7696, 0.9469,
        0.8310, 0.8509, 0.9092, 0.9531, 1.0373, 1.0017, 0.9654, 1.0489, 1.0077,
        1.1116, 0.9957, 0.9957, 1.0720, 1.0431, 1.1447, 1.0489, 1.0373, 1.0777,
        1.1447, 1.2037, 1.1878, 1.1878, 1.1772, 1.2503, 1.2298, 1.1825, 1.1664,
        1.1171, 1.1282, 1.1556, 1.1718, 1.0489, 0.9957, 0.9531, 0.8900, 0.8640,
        0.8310, 0.7130, 0.7202, 0.8575, 0.9155, 1.1171, 0.9092, 1.0431, 1.0137,
        1.0947, 1.1447, 1.1447, 1.0834, 1.0137, 0.9957, 0.9406, 0.8443, 0.7835,
        0.7766, 0.8835, 0.8108, 0.7416, 0.7416, 0.7416, 0.7202, 0.7057, 0.8243,
        0.7835, 0.7486, 0.7972, 0.8175, 0.7556, 0.7416, 0.9344, 0.9092, 0.9028,
        0.9654, 0.9531, 0.9715, 0.8640, 0.8964, 0.8640, 0.9654, 0.8964, 0.8771,
        0.8835, 0.8835, 0.8310, 0.8443, 0.5931, 0.4884, 0.4204, 0.5376, 0.5376,
        0.5931, 0.5457, 0.6316, 0.5696, 0.5376, 0.5295, 0.5132, 0.5050, 0.4290,
        0.3852, 0.3852, 0.3400, 0.385

ip tensor([1.1718, 1.2605, 1.2298, 1.1502, 1.1718, 1.0373, 1.0834, 0.9776, 0.7696,
        0.9469, 0.8310, 0.8509, 0.9092, 0.9531, 1.0373, 1.0017, 0.9654, 1.0489,
        1.0077, 1.1116, 0.9957, 0.9957, 1.0720, 1.0431, 1.1447, 1.0489, 1.0373,
        1.0777, 1.1447, 1.2037, 1.1878, 1.1878, 1.1772, 1.2503, 1.2298, 1.1825,
        1.1664, 1.1171, 1.1282, 1.1556, 1.1718, 1.0489, 0.9957, 0.9531, 0.8900,
        0.8640, 0.8310, 0.7130, 0.7202, 0.8575, 0.9155, 1.1171, 0.9092, 1.0431,
        1.0137, 1.0947, 1.1447, 1.1447, 1.0834, 1.0137, 0.9957, 0.9406, 0.8443,
        0.7835, 0.7766, 0.8835, 0.8108, 0.7416, 0.7416, 0.7416, 0.7202, 0.7057,
        0.8243, 0.7835, 0.7486, 0.7972, 0.8175, 0.7556, 0.7416, 0.9344, 0.9092,
        0.9028, 0.9654, 0.9531, 0.9715, 0.8640, 0.8964, 0.8640, 0.9654, 0.8964,
        0.8771, 0.8835, 0.8835, 0.8310, 0.8443, 0.5931, 0.4884, 0.4204, 0.5376,
        0.5376, 0.5931, 0.5457, 0.6316, 0.5696, 0.5376, 0.5295, 0.5132, 0.5050,
        0.4290, 0.3852, 0.3852, 0.340

In [ ]:
print('train data shape', train_data[0][0].shape)

train data shape torch.Size([192, 5])


# MODEL


In [ ]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        # self.distribution_mu = nn.Linear(params['lstm_hidden_dim'], 1)
        # self.distribution_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        # self.distribution_sigma = nn.Softplus()

        self.price_mu = nn.Sequential(
                      nn.Linear(params['lstm_hidden_dim'],
                                params['num_gmms']),
                      nn.GELU()
                    )
        
        self.price_sigma = nn.Sequential(
                nn.Linear(params['lstm_hidden_dim'],
                          params['num_gmms']),
                nn.Softplus()
              )
        
        self.price_gmm_weights = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'],
                                  params['num_gmms']),
                        nn.Softplus()
                      )


    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        onehot_embed = self.embedding(company_index)
        
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        price_mu = self.price_mu(out1)
        price_sigma = self.price_sigma(out1)
        price_gmm_weights = self.price_gmm_weights(out1)
        
        return price_mu, price_sigma, price_gmm_weights, hn_cn



In [ ]:
model = Network(params)

In [ ]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_mu, out_sigma, out_gmm_weights,  ht_ct = model.forward(x=ip_covariate)

  print('out mu shape', out_mu.shape)
  print('out_sigma shape', out_sigma.shape)
  print('out_gmm_weights', out_gmm_weights.shape)
  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

out mu shape torch.Size([96, 192, 1])
out_sigma shape torch.Size([96, 192, 1])
out_gmm_weights torch.Size([96, 192, 1])
ht shape torch.Size([4, 96, 64]) torch.Size([4, 96, 64])


In [ ]:
def gmm_fit(gmm_mu, gmm_sigma, gmm_mixture_weights):
    mult_gaus = torch.distributions.Normal(gmm_mu, gmm_sigma)
    comp = torch.distributions.Independent(mult_gaus, 1) # The Actual "gaussian" will be the last dim. First 2 dim are Batch, NumMixtures
    mix = torch.distributions.Categorical(gmm_mixture_weights)
    gmms = torch.distributions.mixture_same_family.MixtureSameFamily(mix, comp)
    return gmms


In [ ]:
def gmm_fit_and_sample(price_mu, price_sigma, price_gmm_weights):

  window_len = price_mu.shape[1]
  batch_size = price_mu.shape[0]
  price_mu_reshape = price_mu.view(batch_size * window_len, params['num_gmms'], params['mu_sigma_dim'])
  price_sigma_reshape = price_sigma.view(batch_size * window_len, params['num_gmms'], params['mu_sigma_dim'])
  price_gmm_weights_rehsape = price_gmm_weights.view(batch_size * window_len, params['num_gmms'])

  gmm_prices = gmm_fit(gmm_mu=price_mu_reshape,
                      gmm_sigma=price_sigma_reshape,
                      gmm_mixture_weights=price_gmm_weights_rehsape) 
  
  price_sampled_reshape = gmm_prices.sample()
  price_sampled = price_sampled_reshape.view(batch_size, window_len,  params['mu_sigma_dim'])

  return price_sampled

In [ ]:
def loss_fn_updated(mu, sigma, gmm_weights, labels):

  batch_size, seq_len, _ = mu.shape

  mu = mu.view(batch_size * params['window_size'], params['num_gmms'], params['mu_sigma_dim'])
  sigma = sigma.view(batch_size * params['window_size'], params['num_gmms'], params['mu_sigma_dim'])
  gmm_weights = gmm_weights.view(batch_size * params['window_size'], params['num_gmms'])

  gmm_prices = gmm_fit(gmm_mu=mu,
                       gmm_sigma=sigma,
                       gmm_mixture_weights=gmm_weights) 
  
  labels = labels.view(batch_size * params['window_size'], params['mu_sigma_dim'])

  
  price_loss = -1 * torch.sum(gmm_prices.log_prob(labels)) / (batch_size * params['window_size'])

  return price_loss

In [ ]:
def accuracy_RMSE_train(mu: torch.Tensor, labels: torch.Tensor):
  predictions = mu.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [ ]:
def accuracy_RMSE_test(predictions: torch.Tensor, labels: torch.Tensor):
  predictions = predictions.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [ ]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [ ]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_price_mu = None
  overall_price_sigma = None
  overall_price_gmm_weights = None
  overall_price_label = None  
  overall_price_pred = None


  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):

      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      price_mu, price_sigma, price_gmm_weights, hn_cn = model(x=cond_ip, h0_c0=None)

      price_sampled = gmm_fit_and_sample(price_mu, price_sigma, price_gmm_weights)

      price_sampled_last = price_sampled[:, -1, :].unsqueeze(1)

      batch_price_mu = price_mu
      batch_price_sigma = price_sigma
      batch_price_gmm_weights = price_gmm_weights
      batch_price_pred = price_sampled

      for t in range(0, prediction_period):

        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:,0, 0] = price_sampled_last[:, 0, 0] # Shape -> Bx1x1

        price_pred_mu, price_pred_sigma, price_pred_gmm_weights, hn_cn = model(x=pred_cov_ip, h0_c0=hn_cn)

        price_sampled_last = gmm_fit_and_sample(price_pred_mu, price_pred_sigma, price_pred_gmm_weights)


        batch_price_mu = torch.cat((batch_price_mu, price_pred_mu), dim=1)
        batch_price_sigma = torch.cat((batch_price_sigma, price_pred_sigma), dim=1)
        batch_price_gmm_weights = torch.cat((batch_price_gmm_weights, price_pred_gmm_weights), dim=1)
        batch_price_pred = torch.cat((batch_price_pred, price_sampled_last), dim=1)

      overall_price_mu = batch_price_mu if overall_price_mu is None else torch.cat((overall_price_mu, batch_price_mu), dim=0)
      overall_price_sigma = batch_price_sigma if overall_price_sigma is None else torch.cat((overall_price_sigma, batch_price_sigma), dim=0)
      overall_price_gmm_weights = batch_price_gmm_weights if overall_price_gmm_weights is None else torch.cat((overall_price_gmm_weights, batch_price_gmm_weights), dim=0)
      overall_price_pred = batch_price_pred if overall_price_pred is None else torch.cat((overall_price_pred, batch_price_pred), dim=0)
      overall_price_label = op_label if overall_price_label is None else torch.cat((overall_price_label, op_label), dim=0)



  return overall_price_mu, overall_price_sigma, overall_price_gmm_weights, overall_price_pred, overall_price_label

In [ ]:
# rmse = accuracy_RMSE_train(overall_mu, overall_label)
# print("rmse ", rmse)

In [ ]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])


In [ ]:
overall_price_mu, overall_price_sigma, overall_price_gmm_weights, overall_price_pred, overall_price_label = validate(model)
print("overall_price_mu", overall_price_mu.shape)
print("overall_price_sigma", overall_price_sigma.shape)
print("overall_price_gmm_weights", overall_price_gmm_weights.shape)
print("overall_price_pred", overall_price_pred.shape)
print("overall_price_label", overall_price_label.shape)

overall_price_mu torch.Size([4236, 192, 1])
overall_price_sigma torch.Size([4236, 192, 1])
overall_price_gmm_weights torch.Size([4236, 192, 1])
overall_price_pred torch.Size([4236, 192, 1])
overall_price_label torch.Size([4236, 192])


In [ ]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  print(ip_covariate.shape)
  break

torch.Size([96, 192, 5])


# TRAIN

In [ ]:
model_version='deepar_model_volume_log_v1_alldata.pt'
epochs = params['epochs']

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

print(model)

for epoch in range(0, 50):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0


    for i, (ip_covariate, op_label) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        # print(ip_covariate.shape)
        op_label     = op_label.cuda()

        price_mu, price_sigma, price_gmm_weights, hn_cn = model(x=ip_covariate, h0_c0=None)

        # print('mu shape', mu.shape, 'sigma shape', sigma.shape, 'op label', op_label.shape)
        loss = loss_fn_updated(price_mu, price_sigma, price_gmm_weights, op_label)

        # print("loss", loss)

        total_loss += float(loss)
        loss.backward()
        optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    overall_price_mu, overall_price_sigma, overall_price_gmm_weights, overall_price_pred, overall_price_label = validate(model=model)

    val_rmse = accuracy_RMSE_test(overall_price_pred.squeeze(), overall_price_label)

    scheduler.step()
    
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse)
    # break

Network(
  (embedding): Embedding(5, 256)
  (lstm): LSTM(260, 64, num_layers=4, batch_first=True)
  (price_mu): Sequential(
    (0): Linear(in_features=64, out_features=1, bias=True)
    (1): GELU()
  )
  (price_sigma): Sequential(
    (0): Linear(in_features=64, out_features=1, bias=True)
    (1): Softplus(beta=1, threshold=20)
  )
  (price_gmm_weights): Sequential(
    (0): Linear(in_features=64, out_features=1, bias=True)
    (1): Softplus(beta=1, threshold=20)
  )
)


Epoch 1/20: Train Loss 1.0307, Learning Rate 0.0009
rmse is  1.435026528070901


Epoch 2/20: Train Loss 0.7476, Learning Rate 0.0008
rmse is  1.374867521440569


Epoch 3/20: Train Loss 0.4327, Learning Rate 0.0007
rmse is  1.2130337338663704


Epoch 4/20: Train Loss 0.1821, Learning Rate 0.0007
rmse is  1.1014707244738957


...

Epoch 46/20: Train Loss -0.5385, Learning Rate 0.0000
rmse is  0.9812290908916016


Epoch 47/20: Train Loss -0.5390, Learning Rate 0.0000
rmse is  0.9840688438682829


Epoch 48/20: Train Loss -0.5392, Learning Rate 0.0000
rmse is  0.9809745718685305


Epoch 49/20: Train Loss -0.5400, Learning Rate 0.0000
rmse is  0.9821325485135889


Epoch 50/20: Train Loss -0.5413, Learning Rate 0.0000
rmse is  0.9830163706855759


In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/best_dev_acc_0.030142039248817844_deepar_model_no_volume_v1.pt')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted, sigma, window_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  df['predicted_upper'] = predicted[window_id] 
  df['predcited_lower'] = predicted[window_id]

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [ ]:
# #s for the paper 3500 420
model_no_volume = model 
# checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/deepar_model_no_volume_v1.pt')
# model_no_volume.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
overall_price_mu, overall_price_sigma, overall_price_gmm_weights, overall_price_pred, overall_price_label = validate(model=model)
print("overall_price_pred", overall_price_pred.shape)
print("overall_price_label", overall_price_label.shape)
plot_test_output_data(overall_price_label.cpu().numpy(),overall_price_pred.cpu().numpy(), torch.zeros_like(overall_price_pred), 3789)

overall_price_pred torch.Size([4236, 192, 1])
overall_price_label torch.Size([4236, 192])


In [ ]:
|rmse = accuracy_RMSE_test(mu_no_volume[:, 168:], label_no_volume[:, 168:])
print('rmse ', rmse)

rmse  0.5031645008838738


In [ ]:
# #s for the paper 3500 420 2500 69

for i in range(0, 3500, 70):
  plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), i  )

In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result

print("rmse ", rmse(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

print("nd ", ND(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

rmse  0.06767744374120029
nd  0.057781804
